# <b>Etapas da Metodologia</b>

## Etapas para gerar o grafo de conhecimento:

Carregar os dados dos arquivos JSON:

- Carregar os dados dos currículos dos pesquisadores em um DataFrame cuDF.
- Carregar os dados da Matriz CEIS em um DataFrame cuDF.
- Carregar as relações para biológicos e pequenas moléculas em estruturas de dados Python (listas de dicionários).

Criar o grafo cuGraph:
- Criar um grafo direcionado cuGraph.
- Adicionar nós para cada pesquisador, produto da Matriz CEIS e elemento dos relacionamentos (biológicos e pequenas moléculas).
- Definir atributos para os nós, como tipo de nó ("pesquisador", "produto", "relacionamento"), nome, ID, etc.

Conectar os nós:
- Conectar os nós de pesquisadores aos nós de produtos com base na similaridade entre as áreas de atuação do pesquisador e as áreas de aplicação do produto.
- Conectar os nós de produtos aos nós de relacionamento com base no tipo de produto (biológico ou pequena molécula).
- Conectar os nós de relacionamento entre si, seguindo as relações definidas nos arquivos JSON.

Calcular métricas de grafo:
- Calcular métricas de centralidade, como grau de entrada e saída, para identificar os nós mais importantes no grafo.
- Calcular métricas de caminho, como distância e betweenness centrality, para analisar as relações entre os nós.
- Converter o grafo cuGraph para NetworkX:



## Função de perda adequada para aprendizado não-supervisionado

No contexto de aprendizado não-supervisionado para alinhar competências da ICT aos produtos estratégicos, a escolha da função de perda considerou o objetivo de agrupar pesquisadores com competências semelhantes e identificar as lacunas em relação aos produtos estratégicos. Neste caso, não existe nesse cenário rótulos de classe pré-definidos. Portanto, as funções de perda tradicionais, como MSE ou Cross-Entropy, não são as mais adequadas.

Considerando as opções de abordagem em redes neurais (com KANs, com Fourier e Híbrida) que serão utilizadas nos problemas de aprendizado não-supervisionado, a **Triplet Loss** surge como uma escolha promissora para esse tipo de problema, pois se concentra em aprender embeddings que agrupam nós semelhantes e separam nós distintos, o que se alinha com o objetivo de agrupar pesquisadores com competências semelhantes e identificar lacunas.

O raciocínio sobre a Triplet Loss ser uma boa escolha para o aprendizado não-supervisionado em grafos com o objetivo de alinhar competências se aplica não só para abordagem com KANs mas também, de forma similar, se aplica bem, com algumas adaptações, para as outras abordagens de Rede Neural com Fourier e Rede Neural Híbrida.

### Como aplicar a Triplet Loss

1. **Definir trios:**
    * **Âncora:** Um pesquisador.
    * **Positivo:** Outro pesquisador com competências semelhantes à âncora, idealmente trabalhando em um produto estratégico que requer essas competências.
    * **Negativo:** Um pesquisador com competências diferentes da âncora, preferencialmente que não esteja associado a nenhum produto estratégico que a âncora poderia contribuir.

2. **Calcular perda:**
    * A Triplet Loss busca minimizar a distância entre a âncora e o positivo, e maximizar a distância entre a âncora e o negativo.
    * A função de perda é definida como:
      
<center>L = max(d(âncora, positivo) - d(âncora, negativo) + margem, 0)</center>
      
<center>onde 'd(a, b)' é a distância entre os embeddings de 'a' e 'b', e 'margem' é um hiperparâmetro que define a diferença mínima desejada entre as distâncias.</center>

3. **Amostrar trios:**
    * A escolha dos trios é crucial para o bom desempenho da Triplet Loss.
    * Estratégias de amostragem como "hard negative mining" podem ser utilizadas para selecionar os trios mais informativos, que contribuem mais para o aprendizado.

### Vantagens da Triplet Loss

* **Alinhamento com o objetivo:** A Triplet Loss foca em agrupar nós semelhantes e separar nós distintos, o que é diretamente relevante para o problema de alinhamento de competências.
* **Consideração da estrutura do grafo:** A escolha dos trios pode levar em conta as relações entre os nós no grafo, como a colaboração entre pesquisadores ou a participação em projetos comuns.
* **Flexibilidade:** A Triplet Loss pode ser combinada com diferentes arquiteturas de redes neurais em grafos, como GNNs, para capturar as informações do grafo de forma mais eficiente.


### Implementação da função de perda com Triplet Loss

Para implementar a Triplet Loss, foi utilizada biblioteca `PyTorch Metric Learning`, que oferece diversas funções de perda e métodos de mineração para facilitar o treinamento.

Lembre-se que a escolha da função de perda é apenas um dos aspectos do problema. A arquitetura da rede neural, a estratégia de amostragem dos trios e a escolha dos hiperparâmetros também são importantes para o bom desempenho do modelo.

# <b>Abordagens GNN com função perda Triplet Loss</b>

## Rede KAN com função de perda por Triplet Loss:

A KAN foi responsável por gerar os embeddings dos nós do grafo, que foram utilizados pela Triplet Loss para calcular as distâncias entre os nós. A saída da KAN é um vetor de embedding para cada nó, representando suas características e relações no grafo.

### Definição dos trios:

A definição dos trios (âncora, positivo e negativo) segue a mesma lógica das outras abordagens:
- Âncora: Um pesquisador.
- Positivo: Outro pesquisador com competências semelhantes à âncora, idealmente associado a um produto estratégico que demanda essas competências.
- Negativo: Um pesquisador com competências diferentes da âncora, preferencialmente não associado a produtos estratégicos que a âncora poderia contribuir.


### Cálculo da perda:

- A Triplet Loss foi calculada com base nos embeddings gerados pela Kolmogorov-Arnold Networks (KAN). 

- Quanto à amostragem dos trios, foi utilizada uma estratégia de amostragem eficiente, a "hard negative mining", para selecionar os trios mais informativos para o aprendizado.

- A Triplet Loss foi utilizada para aprender embeddings que representem tanto as características estruturais do grafo quanto as relações de similaridade entre os nós, porém com o diferencial de aprender sobre as relações que ligam os nós (arestas).

- Quanto à otimização, durante o treinamento, o otimizador ajustará os pesos da KAN para minimizar a Triplet Loss. A função de perda buscou minimizar a distância entre a âncora e o positivo, e maximizar a distância entre a âncora e o negativo. Isso gerou embeddings que agrupam pesquisadores com competências semelhantes e separam aqueles com competências distintas.

## Rede Neural Fourier (Transformadas de Fourier + GNN):

* A Triplet Loss pode ser utilizada em conjunto com as Transformadas de Fourier para aprender embeddings que representem tanto as características estruturais do grafo quanto as relações de similaridade entre os nós.

* As Transformadas de Fourier podem auxiliar na identificação de padrões e características relevantes na estrutura do grafo,  enquanto a Triplet Loss guia o aprendizado para agrupar nós com competências semelhantes.

* A combinação dessas técnicas pode levar a um modelo mais robusto e capaz de capturar diferentes aspectos do problema de alinhamento de competências.

## Rede Neural Híbrida (Controle de Sincronização + GNN):

* A Triplet Loss foi utilizada para auxiliar na sincronização dos nós, incentivando o agrupamento de pesquisadores com competências semelhantes e a separação daqueles com competências diferentes.

* A dinâmica de sincronização, baseada no valor de Fiedler, pode complementar a Triplet Loss, ajudando a identificar os nós mais importantes para o alinhamento de competências e guiando o aprendizado da rede.

* A combinação da Triplet Loss com a GNN permite capturar as informações locais e globais do grafo,  enquanto a dinâmica de sincronização fornece uma perspectiva adicional sobre a importância dos nós e arestas.

**Observações Gerais:**

A escolha da função de perda é apenas um dos aspectos do problema. A arquitetura da rede neural, a estratégia de amostragem dos trios e a escolha dos hiperparâmetros também são importantes para o bom desempenho do modelo. Outras funções de perda, como a DGI ou a VGAE, também podem ser exploradas, especialmente se você desejar capturar a estrutura global do grafo e aprender representações mais robustas. É crucial avaliar o desempenho do modelo com diferentes métricas de avaliação, como as discutidas anteriormente, para garantir que ele esteja alinhando as competências da ICT aos produtos estratégicos de forma eficaz.

A Triplet Loss é uma função de perda versátil que pode ser aplicada em diferentes abordagens de redes neurais em grafos. Para o problema de alinhamento de competências em aprendizado não-supervisionado, a escolha da melhor função de perda e da arquitetura do modelo dependerá das características específicas do seu problema e dos seus objetivos, mas pode-se observar de forma geral, que:

* A escolha dos trios (âncora, positivo e negativo) é crucial para o bom desempenho da Triplet Loss. É importante definir uma estratégia de amostragem que leve em conta as características do grafo e o objetivo de alinhamento de competências.
* A margem da Triplet Loss é um hiperparâmetro importante que deve ser ajustado para obter o melhor desempenho.
* A Triplet Loss pode ser combinada com outras funções de perda, como a DGI ou a VGAE, para capturar diferentes aspectos do problema e melhorar o aprendizado das representações.
* É fundamental avaliar o desempenho do modelo com diferentes métricas de avaliação,  além de analisar os embeddings e clusters gerados para garantir que o modelo esteja alinhando as competências de forma eficaz e interpretável.


A interpretabilidade do modelo também é importante. Analise os embeddings gerados e os clusters formados para entender como o modelo está realizando o alinhamento de competências.
Ao combinar a Triplet Loss com uma arquitetura de rede neural em grafos adequada e uma estratégia de amostragem eficiente, você poderá desenvolver um modelo de aprendizado não-supervisionado capaz de alinhar as competências da ICT aos produtos estratégicos e auxiliar na identificação de lacunas e oportunidades de desenvolvimento.

## Parâmetros iniciais e ajuste de hiperparâmetros

A escolha dos parâmetros para cada modelo depende de diversos fatores, como o tamanho e a complexidade do grafo de conhecimento, a quantidade de dados de treinamento, a capacidade computacional disponível e o objetivo da análise. 

Partimos de alguns valores iniciais razoáveis para cada parâmetro para ajustá-los posteriormente com base nos resultados dos experimentos.

* `num_features`: Depende do número de features para cada nó do grafo (pesquisadores, competências e produtos). 
    * **Exemplo:** Se você tiver 10 features para cada pesquisador (e.g., anos de experiência, número de publicações, áreas de atuação), 5 features para cada competência (e.g., nível de proficiência, tipo de competência) e 3 features para cada produto estratégico (e.g., complexidade, área terapêutica), você pode concatenar esses features em um vetor de 18 dimensões.
    * **Valor inicial:**  Número total de features extraídas para cada nó.

* `hidden_dim`: Define a dimensão das camadas ocultas da KAN.
    * **Valor inicial:**  64 ou 128.  Experimentar variações em potências de 2.

* `num_classes`: Define o número de classes (clusters) que você deseja gerar.
    * **Valor inicial:**  Mesmo valor utilizado no modelo híbrido.

* `num_layers`: Define o número de camadas na KAN.
    * **Valor inicial:**  3 ou 4 é um bom início. Escolher um valor que faça sentido para o problema.  Você pode usar técnicas como o método do cotovelo para auxiliar na escolha do número ideal de clusters.

* `dropout`: Define a taxa de dropout.
    * **Valor inicial:**  0.5.

### Observações:

* **Ajuste dos parâmetros:**  É fundamental realizar experimentos e analisar os resultados para ajustar os parâmetros e encontrar a melhor configuração para cada modelo.
* **Técnicas de otimização:**  Utilize técnicas como grid search ou random search para explorar diferentes combinações de parâmetros e encontrar a configuração ótima.
* **Métricas de avaliação:**  Monitore as métricas de avaliação durante o treinamento e a validação para avaliar o desempenho do modelo e guiar o ajuste dos parâmetros.

Estes foram apenas os valores iniciais. A melhor configuração para os parâmetros dependerá das características do grafo de conhecimento, da quantidade de dados de treinamento e do objetivo da análise. Esses valores foram buscados por otimização automatizada de hiperparâmetros dentro de espaços de busca relacionados com os valores inciais mostrados acima. A partir dos desse ajuste e de vários experimentos para medição de desempenho computacional é que se chegou aos hiperparâmetros adequados para obter o melhor desempenho de cada modelo.

# <b>Implementações em código</b>

Para criar a estrutura de manipulação dos arquivos JSON e construir o grafo de análise, primeiro carregamos os dados dos arquivos JSON e, em seguida, criamos o grafo de conhecimento.

In [6]:
# %pip install pytorch-metric-learning

# Importações necessárias
import os
import json
import cudf
import cugraph
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from git import Repo
from pyvis.network import Network
from semantic_matcher import RedeNeuralHibrida, RedeNeuralKAN, RedeNeuralFourier

# 1. Recuperar dados pré-processados
# Informar caminho para arquivo CSV usando raiz do repositório Git como referência
repo = Repo(search_parent_directories=True)
root_folder = repo.working_tree_dir
json_folder = os.path.join(root_folder, '_data', 'out_json')

# Carregar os dados dos arquivos JSON
with open(os.path.join(json_folder,'curriculos.json'), 'r') as f:
    curriculos_data = json.load(f)
with open(os.path.join(json_folder,'matriz_ceis.json'), 'r') as f:
    matriz_ceis_data = json.load(f)
with open(os.path.join(json_folder,'4dm_biologics.json'), 'r') as f:
    relacoes_biologicos = json.load(f)
with open(os.path.join(json_folder,'4dm_smallmolecules.json'), 'r') as f:
    relacoes_pequenas_moleculas = json.load(f)

print("Síntese dos dados para criar o Grafo de Conhecimento:\n")
print("  Relacionamentos da Cadeia de Agregação de Valor em Produtos por tipo:")
print(f"       Biológicos: {len(relacoes_biologicos):2} chaves: {relacoes_biologicos.keys()}")
print(f"    Peq.Moléculas: {len(relacoes_pequenas_moleculas):2} chaves: {relacoes_pequenas_moleculas.keys()}")
print("\n  Dados das entidades em análise:")
print(f"      Matriz_CEIS: {len(matriz_ceis_data.get('blocos')):2} blocos, contendo as chaves: {list(matriz_ceis_data.get('blocos')[0].keys())}")
print(f"       Currículos: {len(curriculos_data):2} currículos, com {len(curriculos_data[0])} chaves em cada currículo")
print(f"\n  Dados sobre cada pesquisador:")
for i in list(curriculos_data[0].keys()):
    print(f"     {i}")

print('\nLista de produtos por Bloco da Matriz CEIS:')
for n,b in enumerate(matriz_ceis_data.get('blocos')):
    print(f"  Bloco: {b.get('titulo')}")
    for p in b.get('produtos'):
        print(f"    {p.get('nome')}")
    print()

Síntese dos dados para criar o Grafo de Conhecimento:

  Relacionamentos da Cadeia de Agregação de Valor em Produtos por tipo:
       Biológicos:  2 chaves: dict_keys(['nodes', 'edges'])
    Peq.Moléculas:  2 chaves: dict_keys(['nodes', 'edges'])

  Dados das entidades em análise:
      Matriz_CEIS:  2 blocos, contendo as chaves: ['bloco', 'id', 'titulo', 'produtos', 'desafios']
       Currículos: 38 currículos, com 15 chaves em cada currículo

  Dados sobre cada pesquisador:
     Identificação
     Idiomas
     Formação
     Atuação Profissional
     Linhas de Pesquisa
     Áreas
     Produções
     ProjetosPesquisa
     ProjetosExtensão
     ProjetosDesenvolvimento
     ProjetosOutros
     Patentes e registros
     Bancas
     Orientações
     JCR2

Lista de produtos por Bloco da Matriz CEIS:
  Bloco: B01. PREPARAÇÃO DO SISTEMA DE SAÚDE PARA EMERGÊNCIAS SANITÁRIAS
    Vacinas do PNI que demandem atualização tecnológica
    Vacinas do PNI que possuem dependência externa
    Vacina vír

In [2]:
print("Lista de relacionamentos por tipo de produtos")
for i,j in relacoes_pequenas_moleculas.items():
    for k in j:
        rotulo = k.get('id')
        if rotulo:
            print(f"  {rotulo}")
        else:
            print(f"{  k}")

Lista de relacionamentos por tipo de produtos
  BasicScience
  BiomarkerDevtProgram
  AssayDevelopmentC
  BiomedicalInformatics
  HypothesisGeneration
  DiseasePathophysiology
  TherapeuticTargets
  DataMining
  NewIndicationRepurposingG
  CellLines
  AnimalModels
  MolecularPathway
  DataRepository
  PrognosticPredictiveBiomarker
  ResponseBiomarker
  HTSAssay
  CompoundLibraries
  ChemInformatics
  HTSSystem
  Genotypes
  Phenotypes
  Qualification
  QualifiedInvestigators
  Hits
  Misses
  Biorepositories
  Biospecimens
  RegistriesEMRs
  CompanionDiagnostics
  SurrogateEndpoint
  ResearcherInvestigator
  IRBReview
  IRBApproval
  UnapprovedCompoundsNCENME
  ClinicalCohorts
  NaturalHistoryEpiStudiesE
  ClinicalTrialPlanningPreparation
  LeadOptimization
  IND
  ClinicalTrialPlanningPreparation
  StudySponsor
  OutcomesRatesEvents
  LeadCompounds
  PopulationSpecific
  Recruitment
  InformedConsent
  RegulatoryComplianceStrategy
  IdentificationOfTargetPopulation
  InSilicoModeling


In [3]:
print("Lista de relacionamentos por tipo de produtos")
for i,j in relacoes_biologicos.items():
    for k in j:
        rotulo = k.get('id')
        if rotulo:
            print(f"  {rotulo}")
        else:
            print(f"{  k}")

Lista de relacionamentos por tipo de produtos
  BasicScience
  BiomedicalInformatics
  DataMining
  DataRepository
  TargetPharmacology
  TargetValidation
  AssayDevelopment
  Hit2Lead
  LeadIdentification
  LeadOptimization
  CMC
  ClinicalResearch
  ClinicalTrials
  RegulatoryReview
  Postmarketing
  PostmarketingStudies
  MedicalLandscape
  MedicalNeed
  ClinicalPracticeGuidelines
  HealthEconomics
  Regulatory
  RegulatoryAgencies
  RegulatoryGuidelines
  Reimbursement
  Payers
  ReimbursementPolicies
  MarketAccess
  Pricing
  Commercialization
  Marketing
  PatientAccess
  Manufacturing
  SupplyChain
  Distribution
{'from': 'BasicScience', 'to': 'BiomedicalInformatics'}
{'from': 'BiomedicalInformatics', 'to': 'DataMining'}
{'from': 'DataMining', 'to': 'DataRepository'}
{'from': 'BasicScience', 'to': 'TargetPharmacology'}
{'from': 'TargetPharmacology', 'to': 'TargetValidation'}
{'from': 'TargetValidation', 'to': 'AssayDevelopment'}
{'from': 'AssayDevelopment', 'to': 'Hit2Lead'}
{'

In [4]:
# print("Lista de relacionamentos por tipo de produtos")
# for i,j in relacoes_biologicos.items():
#     for k in j:
#         print(k.keys())

# 2. Preparar o Grafo de Conhecimento


In [5]:
import networkx as nx

grafo = nx.Graph()
# ... (código para adicionar nós e arestas ao grafo)

grafo_treino, grafo_teste = train_test_split(grafo, test_size=0.2)

# Configuração dos parâmetros dos modelos
parametros_iniciais = {
    'num_features': ...,  # Número de features dos nós
    'hidden_dim': 4,  # Dimensão da camada oculta
    'num_classes': 7,  # Número de classes (clusters)
    'dropout': 0.5  # Taxa de dropout
}

NameError: name 'nx' is not defined

# 3. Criação dos modelos com instanciação das classes

modelo_kan     = RedeNeuralKAN(parametros_iniciais)
modelo_fourier = RedeNeuralFourier(parametros_iniciais)
modelo_hibrido = RedeNeuralHibrida(parametros_iniciais)

# 4. Treinamento dos modelos

Utilizar o grafo de treinamento (grafo_treino) para treinar cada um dos modelos. 

## 4.1. Treinamento do modelo de GNN com Sincronização (híbrido)

    Inspirado em https://iopscience.iop.org/article/10.1209/0295-5075/ad76d6

In [5]:
def treinar_modelo(modelo, grafo_treino, epochs=100, learning_rate=0.01):
    """
    Treina um modelo de rede neural em grafos.

    Args:
      modelo: O modelo da rede neural.
      grafo_treino: O grafo de conhecimento para treinamento.
      epochs: O número de épocas de treinamento.
      learning_rate: A taxa de aprendizado do otimizador.

    Funcionamento:
      Recebe o modelo, o grafo de treinamento, o número de épocas e a taxa de aprendizado como parâmetros.
      Cria um otimizador Adam para atualizar os pesos do modelo.
      Define a função de perda como CrossEntropyLoss (ajustável para a função de perda mais adequada ao problema).
      Prepara os dados de treinamento (features, edge_index, edge_weight e labels, se houver).
      Itera pelas épocas de treinamento, executando o modelo, calculando a perda, os gradientes e atualizando os pesos.
    """
    try:
        # Criar o otimizador
        optimizer = torch.optim.Adam(modelo.parameters(), lr=learning_rate)

        # Definir a função de perda (exemplo: cross-entropy)
        criterion = nn.CrossEntropyLoss()

        # Preparar os dados de treinamento
        x = torch.tensor(list(nx.get_node_attributes(grafo_treino, 'features').values()), dtype=torch.float)
        edge_index = torch.tensor(list(grafo_treino.edges()), dtype=torch.long).t().contiguous()
        edge_weight = torch.tensor(list(nx.get_edge_attributes(grafo_treino, 'weight').values()), dtype=torch.float)
        
        # ... (código para obter os rótulos dos nós, se houver)
        # labels = ...

        # Treinar o modelo
        modelo.train()  # Mudar para o modo de treinamento
        for epoch in range(epochs):
            optimizer.zero_grad()  # Zerar os gradientes
            out = modelo(x, edge_index, edge_weight)  # Executar o modelo
            loss = criterion(out, labels)  # Calcular a perda
            loss.backward()  # Calcular os gradientes
            optimizer.step()  # Atualizar os pesos do modelo

            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

    except Exception as e:
        print(f"Erro no treinamento do modelo: {e}")
        raise e

# Chamar a função para treinar o modelo_hibrido
treinar_modelo(modelo_hibrido, grafo_treino, epochs=100, learning_rate=0.01)

## 4.2 Função de treinamento do modelo_kan_mse

    Com função de perda Mean Squared Error

In [ ]:
criterion_kan = nn.MSELoss()  

def treinar_modelo_kan_mse(modelo, grafo_treino, epochs=100, learning_rate=0.01, criterion=criterion_kan):
    """
    Treina um modelo de rede neural KAN em grafos.

    Args:
      modelo: O modelo da rede neural KAN.
      grafo_treino: O grafo de conhecimento para treinamento.
      epochs: O número de épocas de treinamento.
      learning_rate: A taxa de aprendizado do otimizador.
      criterion: A função de perda.
    
    Função de Perda: A função de perda utilizada foi o MSELoss. 
    
    Pode-se ajustar para uma função de perda mais adequada ao problema, considerando aprendizado não-supervisionado. 
    Uma opção seria utilizar uma função de perda baseada na distância entre os embeddings de nós que deveriam estar no mesmo cluster.
    Ou ainda, pode-se também usar função de perda baseada em Triplet Loss.
    
    Rótulos: Ajustar o código para obter os rótulos dos nós (labels) de acordo com o seu problema. 
    Como o problema é de aprendizado não-supervisionado, os rótulos podem ser definidos com base em alguma heurística ou conhecimento prévio sobre o problema.
    
    Hiperparâmetros: Ajustar o número de épocas (epochs) e a taxa de aprendizado (learning_rate) para obter o melhor desempenho do modelo.    
    """
    try:
        # Criar o otimizador
        optimizer = torch.optim.Adam(modelo.parameters(), lr=learning_rate)

        # Preparar os dados de treinamento
        x = torch.tensor(list(nx.get_node_attributes(grafo_treino, 'features').values()), dtype=torch.float)
        edge_index = torch.tensor(list(grafo_treino.edges()), dtype=torch.long).t().contiguous()
        edge_weight = torch.tensor(list(nx.get_edge_attributes(grafo_treino, 'weight').values()), dtype=torch.float)
        
        # ... (código para obter os rótulos dos nós, se houver - ajustar para o seu problema)
        # labels = ...

        # Treinar o modelo
        modelo.train()  # Mudar para o modo de treinamento
        for epoch in range(epochs):
            optimizer.zero_grad()  # Zerar os gradientes
            out = modelo(x, edge_index, edge_weight)  # Executar o modelo
            
            # Calcular a perda (considerando que labels é um tensor com os valores desejados para os nós)
            loss = criterion(out, labels)  
            loss.backward()  # Calcular os gradientes
            optimizer.step()  # Atualizar os pesos do modelo

            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

    except Exception as e:
        print(f"Erro no treinamento do modelo KAN: {e}")
        raise e

# Chamar a função para treinar o modelo_kan
treinar_modelo_kan_mse(modelo_kan, grafo_treino, epochs=100, learning_rate=0.01)

## 4.3 Função de treinamento do modelo_kan_triplet

    Com função de perda por triplet loss

In [ ]:
from pytorch_metric_learning import losses

# Exemplo de função para gerar trios (âncora, positivo, negativo)
def gerar_triplets(grafo, embeddings):
    """
    Gera trios (âncora, positivo, negativo) para a Triplet Loss.

    Args:
      grafo: O grafo de conhecimento.
      embeddings: Os embeddings dos nós.

    Returns:
      Um tensor com os trios.
    """
    triplets = []
    for no_ancora in grafo.nodes():
        # Encontrar nós positivos (similares à âncora)
        # ... (implementar lógica para encontrar nós positivos)

        # Encontrar nós negativos (diferentes da âncora)
        # ... (implementar lógica para encontrar nós negativos)

        for no_positivo in nos_positivos:
            for no_negativo in nos_negativos:
                triplets.append([no_ancora, no_positivo, no_negativo])

    return torch.tensor(triplets)

def treinar_modelo_kan_triplet(modelo, grafo_treino, epochs=100, learning_rate=0.01):
    """
    Treina um modelo de rede neural KAN em grafos utilizando Triplet Loss.

    Args:
      modelo: O modelo da rede neural KAN.
      grafo_treino: O grafo de conhecimento para treinamento.
      epochs: O número de épocas de treinamento.
      learning_rate: A taxa de aprendizado do otimizador.
    """
    try:
        # Criar o otimizador
        optimizer = torch.optim.Adam(modelo.parameters(), lr=learning_rate)

        # Preparar os dados de treinamento
        x = torch.tensor(list(nx.get_node_attributes(grafo_treino, 'features').values()), dtype=torch.float)
        edge_index = torch.tensor(list(grafo_treino.edges()), dtype=torch.long).t().contiguous()
        edge_weight = torch.tensor(list(nx.get_edge_attributes(grafo_treino, 'weight').values()), dtype=torch.float)

        # Criar a função de perda Triplet Loss
        loss_func = losses.TripletMarginLoss()

        # Treinar o modelo
        modelo.train()  # Mudar para o modo de treinamento
        for epoch in range(epochs):
            optimizer.zero_grad()  # Zerar os gradientes
            embeddings = modelo(x, edge_index, edge_weight)  # Executar o modelo
            
            # Gerar os trios (âncora, positivo, negativo), com lógica de acordo com o problema
            triplets = gerar_triplets(grafo_treino, embeddings) 

            # Calcular a Triplet Loss
            loss = loss_func(embeddings, triplets)  
            loss.backward()  # Calcular os gradientes
            optimizer.step()  # Atualizar os pesos do modelo

            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

    except Exception as e:
        print(f"Erro no treinamento do modelo KAN: {e}")
        raise e

# Chamar a função para treinar o modelo_kan
treinar_modelo_kan_triplet(modelo_kan, grafo_treino, epochs=100, learning_rate=0.01)

## 4.4 Função para treinamento do modelo_fourier

In [ ]:
# Treinar o modelo_fourier
criterion_fourier = nn.KLDivLoss() # Ajustar a função de perda para o modelo Fourier (exemplo: Kullback-Leibler Divergence)  

def treinar_modelo_fourier(modelo, grafo_treino, epochs=100, learning_rate=0.01, criterion=criterion_fourier):
    """
    Treina um modelo de rede neural Fourier em grafos.

    Args:
      modelo: O modelo da rede neural Fourier.
      grafo_treino: O grafo de conhecimento para treinamento.
      epochs: O número de épocas de treinamento.
      learning_rate: A taxa de aprendizado do otimizador.
      criterion: A função de perda.
    """
    try:
        # Criar o otimizador
        optimizer = torch.optim.Adam(modelo.parameters(), lr=learning_rate)

        # Preparar os dados de treinamento
        x = torch.tensor(list(nx.get_node_attributes(grafo_treino, 'features').values()), dtype=torch.float)
        edge_index = torch.tensor(list(grafo_treino.edges()), dtype=torch.long).t().contiguous()
        edge_weight = torch.tensor(list(nx.get_edge_attributes(grafo_treino, 'weight').values()), dtype=torch.float)
        # ... (código para obter os rótulos dos nós, se houver - ajustar para o seu problema)
        # labels = ...

        # Treinar o modelo
        modelo.train()  # Mudar para o modo de treinamento
        for epoch in range(epochs):
            optimizer.zero_grad()  # Zerar os gradientes
            out = modelo(x, edge_index, edge_weight)  # Executar o modelo

            # Ajustar a saída do modelo e os rótulos para a KLDivLoss
            # (a saída deve ser um tensor de probabilidades e os rótulos devem ser um tensor de índices)
            out = torch.nn.functional.log_softmax(out, dim=1)  # Aplicar log_softmax na saída
            # Converter labels para one-hot encoding e aplicar log_softmax (ajuste para o seu problema)
            labels_onehot = torch.nn.functional.one_hot(labels, num_classes=out.shape[1]).float()
            labels_onehot = torch.nn.functional.log_softmax(labels_onehot, dim=1) 

            # Calcular a perda 
            loss = criterion(out, labels_onehot)  
            loss.backward()  # Calcular os gradientes
            optimizer.step()  # Atualizar os pesos do modelo

            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

    except Exception as e:
        print(f"Erro no treinamento do modelo Fourier: {e}")
        raise e

# Chamar a função para treinar o modelo_fourier
treinar_modelo_fourier(modelo_fourier, grafo_treino, epochs=100, learning_rate=0.01)

# 5. Instanciar a classe de testes e realizar as análises:

In [ ]:
# Crie uma instância da classe TesteRedeNeural para avaliar o desempenho de cada modelo no grafo de teste (grafo_teste):
teste_rede_neural = TesteRedeNeural(grafo_teste, parametros_modelo)  # Instanciar a classe de testes

# Controle de Sincronização + GNN
clusters_hibrido, embeddings_hibrido = modelo_hibrido.inferir(grafo_teste)  # Realizar inferência
analises_hibrido = teste_rede_neural.testar_inferencia(clusters_hibrido, embeddings_hibrido)

# KAN + GNN
clusters_kan, embeddings_kan = modelo_kan.inferir(grafo_teste)
analises_kan = teste_rede_neural.testar_inferencia(clusters_kan, embeddings_kan)

# Transformadas de Fourier + GNN
clusters_fourier, embeddings_fourier = modelo_fourier.inferir(grafo_teste)
analises_fourier = teste_rede_neural.testar_inferencia(clusters_fourier, embeddings_fourier)

# 6. Comparação dos resultados
def gerar_tabela_latex(analises):
    """
    Gera uma tabela LaTeX com os resultados das análises.

    Args:
      analises: Um dicionário contendo as análises dos modelos.

    Returns:
      Uma string com o código LaTeX da tabela.
    """
    df = pd.DataFrame(analises)
    df.index = ['Híbrido', 'KAN', 'Fourier']
    latex_table = df.to_latex(float_format="%.3f", caption="Resultados das Análises", label="tab:resultados")
    return latex_table

# Agregar as análises em um dicionário
analises = {
    'Híbrido': analises_hibrido,
    'KAN': analises_kan,
    'Fourier': analises_fourier,
}

# Gerar a tabela LaTeX
tabela_latex = gerar_tabela_latex(analises)

# Exibir a tabela LaTeX
print(tabela_latex)

# Salvar a tabela em um arquivo .tex
with open('resultados_analises.tex', 'w') as f:
    f.write(tabela_latex)

In [ ]:
# Criar uma instância da classe SemanticMatching
from semantic_matcher import ENPreprocessor, SemanticMatcher
tradutor = ENPreprocessor()
matcher = SemanticMatcher(curriculos_data, matriz_ceis_data, relacoes_biologicos, relacoes_pequenas_moleculas)

# Executar as etapas de processamento
matcher.traduzir_nomes_produtos()
matcher.extrair_caracteristicas()
matcher.classificar_produtos()
matcher.conectar_produtos_grafo()

# Imprimir informações sobre os grafos
print("Grafo de Biológicos:")
print("Número de nós:", matcher.biologicos.number_of_nodes())
print("Número de arestas:", matcher.biologicos.number_of_edges())

print("\nGrafo de Pequenas Moléculas:")
print("Número de nós:", matcher.pequenas_moleculas.number_of_nodes())
print("Número de arestas:", matcher.pequenas_moleculas.number_of_edges())

In [ ]:
import os
import cudf
import cugraph
import json
from pyvis.network import Network

json_folder = os.path.join(os.getcwd(),'_data','out_json')

# Carregar os dados dos arquivos JSON com cuDF
curriculos_df = cudf.read_json(os.path.join(json_folder,'curriculos.json'))

In [ ]:
curriculos_df.head()

In [ ]:
matriz_ceis_df.keys()

In [ ]:
import cudf
import cugraph
import json
import time
from pyvis.network import Network
from sklearn.metrics.pairwise import cosine_similarity

# Funções para calcular as transformadas de Fourier
def naive_fourier(x, gridsize=300):
    # ... (implementação da Naive Fourier Transform) ...
    pass  # Substitua pelo código da transformada

def gft(x, edge_index):
    # ... (implementação da Graph Fourier Transform) ...
    pass  # Substitua pelo código da transformada

def wavelet_transform(x, edge_index, num_scales=5):
    # ... (implementação da Wavelet Transform) ...
    pass  # Substitua pelo código da transformada

def fractional_fourier(x, alpha=0.5):
    # ... (implementação da Fractional Fourier Transform) ...
    pass  # Substitua pelo código da transformada

def qft(x):
  """Calcula a QFT de um quaternion."""
  # ... (implementação da Transformada de Fourier Quaterniônica) ...
  pass  # Substitua pelo código da transformada

def conectar_pesquisadores_produtos(graph, curriculos_df, matriz_ceis_df, tipo_transformada):
    """Conecta pesquisadores a produtos com base na similaridade de áreas de atuação."""

    # Extrair áreas de atuação dos pesquisadores e produtos
    pesquisadores_areas = curriculos_df['Áreas'].to_arrow().to_pylist()
    produtos_areas = []
    for bloco in matriz_ceis_df['blocos'].to_arrow().to_pylist():
        for produto in bloco['produtos']:
            produtos_areas.append(produto['nome'])  # Usando o nome do produto como representação da área

    # Converter áreas de atuação em vetores numéricos (ex: usando word embeddings)
    # ... (implementar lógica para converter áreas em vetores) ...

    # Aplicar a transformada de Fourier selecionada
    if tipo_transformada == 'naive':
        pesquisadores_transformados = [naive_fourier(areas) for areas in pesquisadores_areas]
        produtos_transformados = [naive_fourier(areas) for areas in produtos_areas]
    elif tipo_transformada == 'gft':
        # ... (aplicar GFT) ...
        pass
    elif tipo_transformada == 'wavelet':
        # ... (aplicar Wavelet Transform) ...
        pass
    elif tipo_transformada == 'fractional':
        # ... (aplicar Fractional Fourier Transform) ...
        pass
    elif tipo_transformada == 'qft':
        # ... (aplicar QFT) ...
        pass
    else:
        raise ValueError("Tipo de transformada inválido.")

    # Calcular a similaridade de cosseno entre os vetores transformados
    similaridade = cosine_similarity(pesquisadores_transformados, produtos_transformados)

    # Criar as arestas no grafo cuGraph com base na similaridade
    limite_similaridade = 0.8  # Definir um limite para a similaridade
    for i, pesquisador_id in enumerate(pesquisadores_ids):
        for j, produto_id in enumerate(produtos_ids):
            if similaridade[i, j] > limite_similaridade:
                G.add_edge(pesquisador_id, produto_id, weight=similaridade[i, j])

# --- Avaliação das abordagens ---

resultados = {}
tipos_transformadas = ['naive', 'gft', 'wavelet', 'fractional', 'qft']

for tipo_transformada in tipos_transformadas:
    inicio = time.time()

    # Criar um novo grafo para cada tipo de transformada
    G = cugraph.Graph()
    G.add_nodes_from(pesquisadores_ids, tipo='pesquisador')
    G.add_nodes_from(produtos_ids, tipo='produto')

    # Conectar pesquisadores e produtos
    conectar_pesquisadores_produtos(G, curriculos_df, matriz_ceis_df, tipo_transformada)

    fim = time.time()
    tempo_execucao = fim - inicio

    # Calcular as métricas de avaliação
    num_arestas = G.number_of_edges()

    # ... (calcular precisão e recall usando um conjunto de dados de referência) ...

    resultados[tipo_transformada] = {
        'num_arestas': num_arestas,
        'tempo_execucao': tempo_execucao,
        # 'precisao': precisao,
        # 'recall': recall
    }

# Imprimir os resultados
print(resultados)

# --- Visualização (opcional) ---
# ... (criar gráficos com Altair para comparar as métricas) ...

In [ ]:

# Pré-processamento dos dados com cuDF
# (Extrair IDs e informações relevantes para os nós e arestas)
pesquisadores_ids = curriculos_df['Identificação']['ID Lattes'].to_arrow().to_pylist()
produtos_ids = []
for bloco in matriz_ceis_df['blocos'].to_arrow().to_pylist():
    for produto in bloco['produtos']:
        produtos_ids.append(produto['id'])

# Criar um grafo cuGraph
G = cugraph.Graph()

# Adicionar os nós ao grafo cuGraph
G.add_nodes_from(pesquisadores_ids, tipo='pesquisador')
G.add_nodes_from(produtos_ids, tipo='produto')

# Criar as arestas entre pesquisadores e produtos (definir critérios)
# ... (implementar lógica para conectar pesquisadores a produtos usando cuDF) ...
# EXEMPLO: conectar pesquisadores a produtos com base na similaridade de áreas de atuação

# Calcular métricas de grafo com cuGraph
degree_centrality = cugraph.degree_centrality(G)

# Converter o grafo cuGraph para NetworkX
graph_nx = G.to_networkx()

# Criar o grafo PyVis 'net' a partir do grafo NetworkX 'graph_nx'
net = Network(notebook=True, directed=True)
net.from_nx(graph_nx)

# Personalizar a visualização (opcional)
# ... (utilizar as métricas calculadas com cuGraph para definir o tamanho dos nós, cores, etc.) ...

# Mostrar o grafo na célula do Jupyter
net.show("graph.html")

In [ ]:
import os
import json
import networkx as nx

json_folder = os.path.join(os.getcwd(),'_data','out_json')

# Carregar os dados dos arquivos JSON
with open(os.path.join(json_folder,'curriculos.json'), 'r') as f:
    curriculos_data = json.load(f)
with open(os.path.join(json_folder,'matriz_ceis.json'), 'r') as f:
    matriz_ceis_data = json.load(f)

# Criar um grafo direcionado
graph = nx.DiGraph()

# Adicionar os pesquisadores como nós
for pesquisador in curriculos_data:
    graph.add_node(pesquisador['Identificação']['ID Lattes'], tipo='pesquisador', **pesquisador)

# Adicionar os produtos como nós
for bloco in matriz_ceis_data['blocos']:
    for produto in bloco['produtos']:
        graph.add_node(produto['id'], tipo='produto', **produto)

# Função para criar as arestas entre pesquisadores e produtos
def create_edges(graph, pesquisador_node, produto_node):
    """Cria arestas entre pesquisadores e produtos com base em suas propriedades."""
    pesquisador_data = pesquisador_node[1]
    produto_data = produto_node[1]

    # Lógica para conectar pesquisadores a produtos (EXEMPLO)
    # Verificar se as áreas de atuação do pesquisador são relevantes para o produto
    for area in pesquisador_data['Áreas'].values():
        if any(keyword in area for keyword in ["Biotecnologia", "Saúde", "Química", "Biologia"]):
            graph.add_edge(pesquisador_node[0], produto_node[0])
            return  # Criar apenas uma aresta por produto

# Iterar pelos nós e criar as arestas
for pesquisador_node in graph.nodes(data=True):
    if pesquisador_node[1]['tipo'] == 'pesquisador':
        for produto_node in graph.nodes(data=True):
            if produto_node[1]['tipo'] == 'produto':
                create_edges(graph, pesquisador_node, produto_node)

# Imprimir os primeiros 5 nós e arestas
print("Nós:", list(graph.nodes(data=True))[:5])
print("Arestas:", list(graph.edges(data=True))[:5])

# Mostrar o número de nós e arestas
print("Número de nós:", graph.number_of_nodes())
print("Número de arestas:", graph.number_of_edges())

# Mostrar os tipos de nós presentes no grafo
tipos_nos = set(no[1]['tipo'] for no in graph.nodes(data=True))
print("Tipos de nós:", tipos_nos)

# Mostrar as propriedades dos nós
print("Propriedades dos nós:")
for no in graph.nodes(data=True):
    print(no)

# Mostrar a distribuição das arestas entre os nós
# (Exemplo: calcular o grau de cada nó)
graus = dict(graph.degree())
print("Distribuição de graus dos nós:", graus)

In [8]:
# %pip install pyvis

In [ ]:
import networkx as nx
from pyvis.network import Network

net = Network(notebook=True, directed=True)  # notebook=True para exibir no Jupyter, directed=True para grafo direcionado

In [10]:
net.from_nx(graph)

In [ ]:
# Definir opções de layout
net.repulsion(node_distance=200, central_gravity=0.2)

# Configurar a física da visualização
net.show_buttons(filter_=['physics'])  # Mostrar botões para controlar a física

# Definir cores para os nós com base no tipo
for node in net.nodes:
    if node['tipo'] == 'pesquisador':
        node['color'] = 'blue'
    elif node['tipo'] == 'produto':
        node['color'] = 'green'

# Ajustar o tamanho dos nós com base no grau
degrees = dict(graph.degree())
for node in net.nodes:
    node['size'] = degrees[node['id']] * 5  # Aumentar o tamanho proporcionalmente ao grau

# net.show("graph.html")

In [ ]:
from IPython.display import HTML
# Exibir o conteúdo do arquivo HTML na célula do Jupyter
HTML(filename='graph.html')

In [ ]:
# import cudf
# import cugraph
# import json
# import networkx as nx
# from pyvis.network import Network
# from googletrans import Translator
# from sklearn.metrics.pairwise import cosine_similarity

# class SemanticMatching:
#     def __init__(self, curriculos_df, matriz_ceis_data, relacoes_biologicos,
#                  relacoes_pequenas_moleculas):
#         self.curriculos_df = curriculos_df
#         self.matriz_ceis_data = matriz_ceis_data
#         self.relacoes_biologicos = relacoes_biologicos
#         self.relacoes_pequenas_moleculas = relacoes_pequenas_moleculas
#         self.translator = Translator()
#         self.produtos_df = self.criar_dataframe_produtos()
#         self.biologicos, self.pequenas_moleculas = self.criar_grafos_relacionamentos()

#     def criar_dataframe_produtos(self):
#         json_folder = os.path.join(os.getcwd(),'_data','out_json')

#         # Carregar a Matriz CEIS
#         with open(os.path.join(json_folder,'matriz_ceis.json'), 'r') as f:
#             matriz_ceis_data = json.load(f)

#         # Extrair os dados dos produtos
#         produtos = []
#         for bloco in matriz_ceis_data['blocos']:
#             for produto in bloco['produtos']:
#                 produto['bloco_id'] = bloco['id']
#                 produto['bloco_nome'] = bloco['titulo']
#                 produtos.append(produto)

#         # Retornar o DataFrame cuDF
#         return cudf.DataFrame(produtos)

#     def criar_grafos_relacionamentos(self):
#         # ... (código para criar os grafos de biológicos e pequenas moléculas) ...
#         biologicos = nx.DiGraph()
#         for node in self.relacoes_biologicos["nodes"]:
#             biologicos.add_node(node['id'], **node)
#         for edge in self.relacoes_biologicos["edges"]:
#             biologicos.add_edge(edge['from'], edge['to'])

#         pequenas_moleculas = nx.DiGraph()
#         for node in self.relacoes_pequenas_moleculas["nodes"]:
#             pequenas_moleculas.add_node(node['id'], **node)
#         for edge in self.relacoes_pequenas_moleculas["edges"]:
#             pequenas_moleculas.add_edge(edge['from'], edge['to'])
#         return biologicos, pequenas_moleculas

#     def traduzir_nomes_produtos(self):
#         # ... (código para traduzir os nomes dos produtos) ...
#         self.produtos_df['nome_en'] = self.produtos_df['nome'].apply(
#             lambda x: self.translator.translate(x, dest='en').text)

#     def extrair_caracteristicas(self):
#         # ... (código para extrair características semânticas) ...
#         pass  # Implemente a extração de características aqui

#     def classificar_produtos(self):
#         # ... (código para classificar os produtos) ...
#         pass  # Implemente a classificação dos produtos aqui

#     def calcular_similaridade(self, produto, grafo, tipo_transformada):
#         # ... (código para calcular similaridade usando a abordagem especificada) ...
#         pass  # Implemente o cálculo de similaridade aqui

#     def conectar_produtos_grafo(self):
#         # ... (código para conectar os produtos aos grafos) ...
#         pass  # Implemente a conexão dos produtos aos grafos aqui

#     def avaliar_desempenho(self):
#         # ... (código para avaliar o desempenho das abordagens) ...
#         pass  # Implemente a avaliação de desempenho aqui